In [22]:
%load_ext autoreload
%autoreload 2

from dataloader import WICO
import tensorflow as tf
from model import Net
from spektral.data import DisjointLoader
from tensorflow import keras

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
wico_dataset = WICO("./dataset/WICO/")
loader = DisjointLoader(wico_dataset, batch_size=1)

In [20]:
model = Net(num_classes=3)

In [23]:
out_log = []

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    for (features, adj, _), y_true in loader:
        with tf.GradientTape() as tape:
            # model prediction
            out, _, _ = model([features, adj])
            
            y_pred = tf.nn.softmax(out, axis=1)
            loss_val = loss_object(y_true=y_true, y_pred=y_pred)

        loss_all.append(loss_val.numpy())
        grads = tape.gradient(loss_val, model.trainable_variables)

        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        



tf.Tensor(1.0990027, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.0990027, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.0990027, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.0966558, shape=(), dtype=float32)
tf.Tensor(1.0966558, shape=(), dtype=float32)
tf.Tensor(1.0990027, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.0966558, shape=(), dtype=float32)
tf.Tensor(1.0966558, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.0990027, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dtype=float32)
tf.Tensor(1.0966558, shape=(), dtype=float32)
tf.Tensor(1.0966558, shape=(), dtype=float32)
tf.Tensor(1.0990027, shape=(), dtype=float32)
tf.Tensor(1.0990027, shape=(), dtype=float32)
tf.Tensor(1.1001817, shape=(), dty

KeyboardInterrupt: 